In [1]:
import pandas as pd
import numpy as np
import requests

from bs4 import BeautifulSoup


url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
Canada_data = BeautifulSoup(source, 'lxml')




In [2]:
columns = ['PostalCode', 'Borough', 'Neighborhood']
toronto_data = pd.DataFrame(columns=columns)
toronto_data




,PostalCode,Borough,Neighborhood


In [12]:
column_names = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)
content = Canada_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0
for tr in table.find_all('tr'):
    i = 0  
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2:
            neighborhood = td.text.strip('\\n').replace(']','')
        toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

# clean dataframe 

toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
      if toronto.iloc[i][2] == 'Not assigned':
            toronto.iloc[i][2] = toronto.iloc[i][1]
            i = i+1                                

            
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.shape

(191, 3)

In [13]:
df = df.dropna()
empty = 'Not assigned'
df = df[(df.Postalcode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]

In [14]:
def neighborhood_list(grouped):
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))

grp = df.groupby(['Postalcode', 'Borough'])
df2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')
df2.shape



(191, 3)

In [22]:
import numpy as np
import pandas as pd

df_lon_lat = pd.read_csv("https://cocl.us/Geospatial_data")
df_lon_lat.rename(columns={'Postal Code':'Postalcode'}, inplace=True)
df_lon_lat.head() 

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [23]:
Toronto_df = pd.merge(df2, df_lon_lat[['Postalcode','Latitude', 'Longitude']], on=['Postalcode'])
Toronto_df

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Etobicoke,Islington Avenue\n,43.806686,-79.194353
1,M1B,Scarborough,"Islington Avenue\n, Rouge\n, Rouge\n, Rouge\n,...",43.806686,-79.194353
2,M1C,Etobicoke,West Deane Park\n,43.784535,-79.160497
3,M1C,Scarborough,"West Deane Park\n, Highland Creek\n, Highland ...",43.784535,-79.160497
4,M1E,Etobicoke,Old Burnhamthorpe\n,43.763573,-79.188711
5,M1E,Scarborough,"Old Burnhamthorpe\n, Guildwood\n, Guildwood\n,...",43.763573,-79.188711
6,M1G,Scarborough,"Not assigned\n, Woburn\n",43.770992,-79.216917
7,M1H,Scarborough,"Not assigned\n, Cedarbrae\n",43.773136,-79.239476
8,M1J,Scarborough,"Not assigned\n, Scarborough Village\n",43.744734,-79.239476
9,M1K,Scarborough,"Not assigned\n, East Birchmount Park\n, East B...",43.727929,-79.262029
